In [15]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from transformers import LLaMAForCausalLM, LLaMATokenizer
import torch

In [3]:
from bragi.verse_parsers import BaseParsedVerseHandler

/root/.pyenv/versions/3.10.10/bin/python


In [3]:
import poesy

In [2]:
CACHE_DIR = 'weights'
SEP = "<sep>"


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = LLaMAForCausalLM.from_pretrained("/src/weights", cache_dir=CACHE_DIR, local_files_only=True)


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [10]:
path = "/src/weights"

In [5]:
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# amodel = LLaMAForCausalLM.from_pretrained(path, cache_dir=CACHE_DIR, local_files_only=True)
model.to(device)


NameError: name 'path' is not defined

In [11]:
tokenizer = LLaMATokenizer.from_pretrained(path, cache_dir=CACHE_DIR, local_files_only=True)

In [13]:
prompt = "Happy birthday to"

In [18]:
model.device

device(type='cuda', index=0)

In [21]:
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

num_beams = 5

outputs = model.generate(
    input_ids,
    num_beams=num_beams,
    num_return_sequences=1,
    no_repeat_ngram_size=1,
    remove_invalid_values=True,
    # logits_processor=processors,
    do_sample=True
)


In [22]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

 Happy birthday to you! I hope it's a great one.
Happy B
